Important Libraries

In [ ]:
!apt-get install -qq git

In [ ]:
# Cloning the repo
!git clone https://github.com/mimisheen/test.git

Cloning into 'test'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), 19.96 KiB | 9.98 MiB/s, done.


In [ ]:
from IPython.display import clear_output
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
!pip install opencv-contrib-python==4.5.5.64
!pip install pydub
!pip install -U openai-whisper
!apt-get install ffmpeg
!pip install gtts
!apt-get update && apt-get install -y libportaudio2
!pip install --upgrade sounddevice
clear_output()

In [ ]:
import time
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
import base64
from base64 import b64decode
from PIL import Image
import os, sys, cv2
import io
from io import BytesIO
import numpy as np
from pathlib import Path
from google.colab.patches import cv2_imshow
from gtts import gTTS
import sounddevice as sd
import soundfile as sf
from gtts.tokenizer import pre_processors, tokenizer_cases, Tokenizer
import whisper

In [ ]:
#Required to mount drive to save images and audio files.
'''
from google.colab import drive
drive.mount('/content/drive')
'''

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#Use this command if you have your own database of images and audio files.
#!cp -r /content/drive/MyDrive/<YOUR_database_folder_name> /content/database

# Main UI for video, audio and transcription (Javascript)

This is the html css code integrated in google colab environment. You will notice html css is integrated in javascript code. This is because you cannot directly access your local webcam or microphone in google colab environment.

In [ ]:
# HTML and JavaScript code to access webcam, capture photo, and record audio
html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Webcam and Audio Recorder</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      margin: 0;
      padding: 20px;
      background-color: #f4f4f4;
    }
    .container {
      max-width: 800px;
      margin: 0 auto;
      padding: 20px;
      background-color: white;
      border-radius: 10px;
      box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
    }
    .video-container {
      position: relative;
      width: 100%;
      height: 480px;
      background-color: black;
      margin-bottom: 20px;
      display: flex;
      justify-content: center;
      align-items: center;
    }

    video {
      width: 100%;
      height: 100%;
      object-fit: cover;
      background-color: black;
      z-index: 1;
    }

    canvas {
      position: absolute;
      top: 0;
      left: 0;
      width: 100%;
      height: 100%;
      z-index: 2;
      pointer-events: none;
    }

    #personNameContainer {
      display: none;
    }
    textarea {
      position: relative;
      width: 100%;
      height: 100px;
      padding: 10px;
      border-radius: 5px;
      border: 1px solid #ccc;
      font-size: 16px;
      resize: none;
      box-sizing: border-box;
    }
    #status {
      margin-top: 10px;
    }
    .name-tag {
        padding: 10px 20px;
        background-color: #4caf50;
        color: white;
        border-radius: 5px;
        font-size: 16px;
        font-weight: bold;
    }
  </style>
</head>
<body>
  <div class="container">
    <h1>Webcam Capture and Audio Recorder</h1>

    <!-- Video element for webcam feed -->
    <video id="webcamVideo" autoplay playsinline muted></video>
    <img id="overlayImg" style="position: absolute; top: 0; left: 0;" />
    <div id="labelDisplay" style="position: absolute; top: 10px; left: 10px; color: white; background: rgba(0,0,0,0.5); padding: 5px;"></div>
    <div id="errorMsg"></div>
    <div id="debugOutput"></div>
    <div id="statusElement"></div>

    <!-- Input for person's name (Initially hidden) -->
    <div id="personNameContainer">
      <h3>Enter Person's Name:</h3>
      <input type="text" id="personName" placeholder="Enter Name" />
      <br><br>
      <button onclick="startCapture()">Start Capturing</button>
    </div>

    <canvas id="captureCanvas"></canvas>

    <!-- Buttons for Webcam -->
    <button id="startWebcamBtn" onclick="startWebcam()">Start Webcam</button>
    <button id="stopWebcamBtn" onclick="stopWebcam()">Stop Webcam</button>
    <button id="captureBtn" onclick="showNameInput()">Capture Photo</button>

    <!-- Audio Recorder Buttons -->
    <h3>Audio Recorder</h3>
    <button id="startAudioBtn">Start Recording</button>
    <button id="stopAudioBtn">Stop Recording</button>
    <p id="audioStatus"></p>

    <p id="status"></p>

    <!-- Text box -->
    <h3 style="display: flex; align-items: center; gap: 10px;">
      Transcription:
      <span id="detectedLanguageContainer" style="font-size: 14px; color: #555; padding: 5px 10px; background-color: #f0f0f0; border-radius: 5px;">
        Detected Language: <span id="detectedLanguageBox">None</span>
      </span>
    </h3>
    <textarea id="transcriptionBox" placeholder="Transcription will appear here..."></textarea>
    <!-- Save Text Button -->
    <button id="saveTextBtn" onclick="saveText()">Save Text</button>
  </div>

  <script>
    // Variable Declarations (Global Scope)
    let stream = null;
    let captureCount = 0;
    const maxCaptures = 50; // Set to 50 captures
    const captureInterval = 2000; // 2000 ms (2 seconds)
    let frameCount = 0;
    let modelNames = {};
    let frameCounter = 0; // Global frame counter
    let audioStream = null;
    const videoElement = document.getElementById('webcamVideo');
    const captureCanvas = document.getElementById("captureCanvas");
    const statusElement = document.getElementById('statusElement'); // Properly initialized once
    const personNameInput = document.getElementById('personName');
    const personNameContainer = document.getElementById('personNameContainer');
    const captureBtn = document.getElementById('captureBtn');
    const audioStatusElement = document.getElementById('audioStatus');

    let mediaRecorder, audioRecorder, chunks = [], audioChunks = [], capturing = false;

    // Function to Start Webcam
    function startWebcam() {
        if (navigator.mediaDevices.getUserMedia) {
            navigator.mediaDevices.getUserMedia({ video: true })
                .then(function(localStream) {
                    stream = localStream;
                    videoElement.srcObject = stream;
                    statusElement.innerText = "Webcam is running...";

                    videoElement.addEventListener("loadedmetadata", () => {
                        // Start capturing frames

                        captureFrames();
                    });
                })
                .catch(function(err) {
                    console.log("Error: " + err);
                    alert("Unable to access webcam: " + err);
                });
        } else {
            alert('getUserMedia is not supported in this browser.');
        }
    }

    // Function to Capture Frames from Webcam
    async function captureFrames() {
        const captureCanvas = document.createElement("canvas");
        const ctx = captureCanvas.getContext("2d");

        captureCanvas.width = videoElement.videoWidth;
        captureCanvas.height = videoElement.videoHeight;

        const processDuration = 5000; // 5 seconds in milliseconds
        let startTime = Date.now(); // Record the start time for processing

        async function processFrame() {
            const currentTime = Date.now();
            if (currentTime - startTime > processDuration) {
                console.log("Finished processing 5 seconds of frames.");
                //statusElement.innerText = "Processing complete.";
                return; // Stop processing after 5 seconds
            }

            // Draw the video frame to the canvas
            ctx.drawImage(videoElement, 0, 0, videoElement.videoWidth, videoElement.videoHeight);
            const imageDataUrl = captureCanvas.toDataURL("image/jpeg", 0.8);

            try {
                // Send the captured frame to Python for processing via pred_name() function
                await google.colab.kernel.invokeFunction("pred", [imageDataUrl],{});

            } catch (error) {
                console.error("Error calling Python function:", error);
                statusElement.innerText = "Error calling prediction function.";
            }

            // Request the next animation frame to keep processing
            requestAnimationFrame(processFrame);
        }

        processFrame(); // Start the frame processing loop
    }


////////////////////////////////////////////////////////////////////////////////////////
// Function to stop the webcam
    function stopWebcam() {
      if (stream) {
        stream.getTracks().forEach(track => track.stop());
        videoElement.srcObject = null;
        statusElement.innerText = "Webcam stopped.";
      }
    }

////////////////////////////////////////////////////////////////////////////////////////

    async function captureImage(personName) {
      const video = document.getElementById("webcamVideo");
      const canvas = document.createElement("canvas");
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      const ctx = canvas.getContext("2d");
      ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
      const imageDataUrl = canvas.toDataURL("image/jpeg", 0.8);

      // Call Python function to save and process image
      google.colab.kernel.invokeFunction("save_and_process_image", [imageDataUrl, captureCount, personName], {});
    }
////////////////////////////////////////////////////////////////////////////////////////
    // Function to show the name input field and Capture button
    function showNameInput() {
      captureBtn.style.display = 'none';  // Hide Capture button
      personNameContainer.style.display = 'block';  // Show name input field
    }
////////////////////////////////////////////////////////////////////////////////////////
    function startCapture() {
      const personName = personNameInput.value.trim();
      if (personName === "") {
        alert("Please enter the person's name.");
        return;
      }
      captureCount = 0;

      const intervalId = setInterval(() => {
        if (captureCount >= maxCaptures) {
          clearInterval(intervalId);
          document.getElementById("status").innerText = "Capture complete.";
          // Begin training after capturing all images
          google.colab.kernel.invokeFunction("train_model", [], {}).then(() => {
            document.getElementById("status").innerText = "Training completed.";
          }).catch((error) => {
            document.getElementById("status").innerText = "Training failed.";
            console.error(error);
          });
        } else {
          document.getElementById("status").innerText = `Capturing... ${captureCount}/50`;
          captureImage(personName);
          captureCount++;
        }
      }, captureInterval);
    }
////////////////////////////////////////////////////////////////////////////////////////

    // Audio Recorder JavaScript
    document.getElementById('startAudioBtn').addEventListener('click', async () => {
      audioStream = await navigator.mediaDevices.getUserMedia({ audio: true });
      audioRecorder = new MediaRecorder(audioStream);
      audioChunks = [];

      audioRecorder.ondataavailable = event => {
        audioChunks.push(event.data);
      };

      audioRecorder.onstop = async () => {
        const blob = new Blob(audioChunks, { type: 'audio/wav' });
        const reader = new FileReader();
        reader.onloadend = function() {
          const base64data = reader.result.split(',')[1];
          google.colab.kernel.invokeFunction('notebook.save_audio', [base64data, 0], {});
        };
        reader.readAsDataURL(blob);
        audioStatusElement.innerText = "Audio recording stopped. Audio saved.";
      };

      audioRecorder.start();
      audioStatusElement.innerText = "Recording audio...";
    });

    document.getElementById('stopAudioBtn').addEventListener('click', () => {
      audioRecorder.stop();
      audioStream.getTracks().forEach(track => track.stop());
    });

      // Function to save the transcription text when user clicks "Save Text"
    function saveText() {
      const transcriptionText = document.getElementById('transcriptionBox').value;

      // Call the Python function to save the text
      google.colab.kernel.invokeFunction(
        'notebook.save_text',  // Callback registered in Python
        [transcriptionText],  // Arguments passed to Python function
        {}  // Optional callbacks
      ).then(() => {
        // Alert user and clear the text box
        alert("Text saved successfully.");
        document.getElementById('transcriptionBox').value = '';  // Clear the text box
      }).catch((error) => {
        alert("Error saving text: " + error);
      });
    }
  </script>

</body>
</html>
"""


This is the python backend code that is linked with javascript code.

# Image processing (Face recognition)

In [ ]:
# Function to decode the image from base64 and save both original and processed images
def save_and_process_image(data_url, img_count, person_name):
    # Create the person's directory only if it doesn't exist
    save_path = Path("database") / person_name
    if not save_path.exists():
        save_path.mkdir(parents=True, exist_ok=True)
        print(f"Directory created for {person_name} at {save_path}")

    # Decode base64 image
    header, encoded = data_url.split(",", 1)
    binary_data = base64.b64decode(encoded)
    img = Image.open(BytesIO(binary_data))

    # Convert to OpenCV format
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    # Convert image to grayscale for face detection
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

    # Load the Haar cascade classifier for face detection
    haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw bounding boxes around faces in the original image
    for (x, y, w, h) in faces:
        cv2.rectangle(img_cv, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Save the processed image with bounding boxes
    processed_filename = save_path / f"processed_{img_count}.jpg"
    cv2.imwrite(str(processed_filename), img_cv)
    #print(f"Processed image with bounding boxes saved as {processed_filename}")

Training

In [ ]:
def train_model():
    try:
        # Path to the face detection cascade
        haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'  # Ensure correct path

        # Path to the database folder containing subdirectories of face images
        datasets = 'database'  # Replace with your actual dataset path
        print('Training...')

        # Initialize lists to store images, labels, and names
        images, labels, names = [], [], {}
        id = 0

        # Walk through the dataset directory and collect images and corresponding labels
        for subdirs, dirs, files in os.walk(datasets):
            for subdir in dirs:
                if subdir != '.ipynb_checkpoints':  # Exclude notebook checkpoint folders
                    print(f"Processing directory: {subdir}")
                    names[id] = subdir  # Assign a name to the ID
                    subjectpath = os.path.join(datasets, subdir)
                    for filename in os.listdir(subjectpath):
                        path = os.path.join(subjectpath, filename)

                        # Read image and convert to grayscale
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        if img is not None and filename.lower() != '.ipynb_checkpoints':
                            # Resize the image to a fixed size
                            img_resized = cv2.resize(img, (130, 100))
                            images.append(img_resized)  # Append resized image
                            labels.append(id)  # Append corresponding label
                        else:
                            print(f"Skipping file: {filename}")
                    id += 1

        # Ensure we have images for training
        if len(images) == 0:
            raise Exception("No images found for training.")

        # Convert images and labels lists to NumPy arrays for model training
        images = np.array(images)
        labels = np.array(labels)

        # Create and train the face recognizer model
        model = cv2.face.LBPHFaceRecognizer_create()
        model.train(images, labels)
        # Save the model to a file after training
        model.save('face_recognizer.yml')

        # Print training information
        print(f"Training complete with {len(names)} faces.")
        print(f"Trained on: {names}")

        # Return the trained model and the names dictionary
        return model, names

    except Exception as e:
        print(f"Training failed with error: {e}")
        return None, None



Prediction

In [ ]:
model, names =train_model()
import cv2
import numpy as np
import base64

# Example of how to load a model (make sure the model is properly trained before)
model = cv2.face.LBPHFaceRecognizer_create()
model.read("face_recognizer.yml")  # Load your trained model

def predict(image_data):
    try:
        # Decode Base64 (handling the comma in the base64 string)
        if "," in image_data:
            base64_str = image_data.split(",")[1]
        else:
            raise ValueError("Invalid image data format")

        # Decode base64 string to numpy array
        nparr = np.frombuffer(base64.b64decode(base64_str), np.uint8)

        # Decode the numpy array to an image
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        if img is None:
            raise ValueError("Decoded image is empty.")

        # Convert the image to grayscale for face detection
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Load the Haar cascade for face detection
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Initialize list for detected labels
        detected_labels = []

        # Process each detected face
        for (x, y, w, h) in faces:
            # Extract the face region
            face = gray_image[y:y+h, x:x+w]

            # Resize the face to the required size for the model
            resized_face = cv2.resize(face, (150, 150))

            # Perform prediction using the trained model
            label_id, confidence = model.predict(resized_face)

            # Retrieve the name corresponding to the label_id
            name = names.get(label_id, "Unknown")  # Default to "Unknown" if label_id not found

            # Append the detected name to the list
            detected_labels.append(name)

        # If no faces detected, add "unknown" to the labels
        if not detected_labels:
            detected_labels.append("unknown")

        # Return the detected labels
        return detected_labels

    except Exception as e:
        print(f"Error in processing image: {e}")
        return []

# Example of how you can call the predict function and get the detected names
def pred(image_data):
    # Call the predict function and get the detected names
    detected_names = predict(image_data)

    # Print or process the detected names outside the predict function
    if detected_names:
        print("Detected names:", ", ".join(detected_names))
    else:
        print("No faces detected or error occurred.")

    # Generate JavaScript to update the status element in the browser with the names
    detected_names_str = ", ".join(detected_names)
    js_code = f"""
    var statusElement = document.getElementById('statusElement');
    if (statusElement) {{
        statusElement.innerText = 'Detected: {detected_names_str}';
    }} else {{
        console.error("Element with id 'statusElement' not found.");
    }}
    """
    display(Javascript(js_code))


# Face recognition evaluation

In [ ]:
!pip install opencv-python
!pip install numpy
!pip install scikit-learn
import cv2
import os
import numpy as np
import base64
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Function to evaluate the face recognition system
def evaluate_face_recognition(model, names, test_images, test_labels):
    predictions = []

    for image_data, true_label in zip(test_images, test_labels):
        # Read the image from the path
        image = cv2.imread(image_data, cv2.IMREAD_COLOR)
        # Encode the image to base64
        retval, buffer = cv2.imencode('.jpg', image)
        image_data = base64.b64encode(buffer).decode('utf-8')

        result = predict(image_data) # Pass only image_data to predict

        if result['status'] == 'success' and len(result['names']) > 0:
            predicted_name = result['names'][0]
        else:
            predicted_name = "Unknown"

        predicted_label = [label for label, name in names.items() if name == predicted_name]
        if predicted_label:
            predictions.append(predicted_label[0])
        else:
            predictions.append(-1)  # Unknown faces

    accuracy = accuracy_score(test_labels, predictions)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    precision = precision_score(test_labels, predictions, average='weighted', zero_division=0)
    recall = recall_score(test_labels, predictions, average='weighted', zero_division=0)
    f1 = f1_score(test_labels, predictions, average='weighted', zero_division=0)

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

'''
Here mention the file paths of the test images and their corresponding labels.

# Example test data (replace with actual base64 images and corresponding labels)
test_images = [
    "/content/database/image1.jpg",
    "/content/database/image2.jpg",
    "/content/database/image3.jpg",
    "/content/database/image4.jpg",
    "/content/database/image5.jpg",
    "/content/database/image6.jpg",
    "/content/database/image7.jpg",
    "/content/database/image8.jpg",
    "/content/database/image9.jpg",
    "/content/database/image10.jpg",
]'''

test_labels = [0]  # Replace with the actual label IDs corresponding to test images

# Train the model
model, names = train_model()

# Evaluate the model on test data
if model and names:
    evaluate_face_recognition(model, names, test_images, test_labels)

Here we are calculating the evaluation metrics for model accuracy in terms of face recognition detection.

In [ ]:
import cv2
import numpy as np
import os
from sklearn.metrics import accuracy_score

# Function to calculate accuracy of the LBPH model
def calculate_accuracy(model, test_images_path, names):
    correct_predictions = 0
    total_images = 0

    # Walk through the test images directory
    for subdir, dirs, files in os.walk(test_images_path):
        for filename in files:
            # Check if the file is an image (e.g., jpg, png, jpeg) and not '.ipynb_checkpoints'
            if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
                # Load the image
                img_path = os.path.join(subdir, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                if img is not None:
                    # Resize the image to match the input size of the model
                    resized_img = cv2.resize(img, (150, 150))

                    # Perform prediction
                    label_id, confidence = model.predict(resized_img)

                    # Get the true label (assumed to be the folder name)
                    true_label = os.path.basename(subdir)

                    # Compare the predicted label to the true label
                    if names.get(label_id) == true_label:
                        correct_predictions += 1

                    total_images += 1
                else:
                    print(f"Failed to load image: {img_path}")

    # Calculate accuracy
    accuracy = (correct_predictions / total_images) * 100 if total_images else 0
    return accuracy

# Example usage
test_images_path = "/content/database"
model = cv2.face.LBPHFaceRecognizer_create()
model.read("face_recognizer.yml")  # Load the trained model
accuracy = calculate_accuracy(model, test_images_path, names)
print(f"Accuracy: {accuracy:.2f}%")


# **Audio Processing and Transcription**

In [ ]:
# Function to get full language name from Whisper's language code
def get_language_name(language_code):
    language_list = [(code, name) for code, name in whisper.tokenizer.LANGUAGES.items()]
    for code, name in language_list:
        if code == language_code:
            return name
    return "Unknown Language"

# Function to preprocess text for TTS
def preprocess_text(text):
    text = pre_processors.abbreviations(text)
    text = pre_processors.end_of_line(text)
    text = pre_processors.tone_marks(text)
    return text

# Function to tokenize and minimize the text
def tokenize_and_minimize(text):
    tokenizer = Tokenizer([
        tokenizer_cases.tone_marks,
        tokenizer_cases.period_comma,
        tokenizer_cases.colon,
        tokenizer_cases.other_punctuation
    ])

    tokens = tokenizer.run(text)

    minimized_tokens = []
    for token in tokens:
        if not token.strip():
            continue
        while len(token) > 100:
            split_index = token.rfind(' ', 0, 100)
            if split_index == -1:
                split_index = 100
            minimized_tokens.append(token[:split_index])
            token = token[split_index:].lstrip()
        minimized_tokens.append(token)

    return minimized_tokens

# Function to generate TTS and play the audio
def generate_and_play_audio(text):
    # Preprocess and tokenize the text
    processed_text = preprocess_text(text)
    tokens = tokenize_and_minimize(processed_text)

    # Create an in-memory file to store audio data
    fp = io.BytesIO()

    # Process each token and generate speech
    for token in tokens:
        if token.strip():
            print(f"Processing token: '{token}'")
            tts = gTTS(text=token, lang='en', slow=False)
            tts.write_to_fp(fp)

    fp.seek(0)  # Reset the file pointer to the start

    # Convert audio data to base64
    audio_base64 = base64.b64encode(fp.read()).decode('utf-8')

    # Create JavaScript code to play audio
    js_code = f"""
        var audio = new Audio("data:audio/wav;base64,{audio_base64}");
        audio.play();
    """
    # Execute JavaScript in Colab (or a browser)
    display(Javascript(js_code))

# Function to save audio file and transcribe it using Whisper API
def save_audio(base64data, count):
  try:
    audio_data = b64decode(base64data)
    audio_filename = f"recording_{count}.wav"

    with open(audio_filename, 'wb') as f:
        f.write(audio_data)

# Load the Whisper model
    model = whisper.load_model("turbo")

    # Transcribe the audio file
    result = model.transcribe(audio_filename)
    # Extract the detected language
    detected_language_code = result["language"]
    # Get the full language name
    detected_language_name = get_language_name(detected_language_code)

    # Extract the original transcription
    original_transcription = result["text"]
    # Translate the transcription to English
    translated_result = model.transcribe(audio_filename, language="en")  # Using English language code
    english_translation = translated_result["text"]

    # Save the transcription to a text file
    transcription_filename = f"transcription_{count}.txt"
    with open(transcription_filename, 'w') as f:
        f.write(result["text"])
    # Now, read the content of the transcription file into the 'transcription' variable
    with open(transcription_filename, 'r') as f:
        transcription = f.read()
    # Send the transcription to the text box and the detected language to the inline container
    output.eval_js(f"""
        document.getElementById('transcriptionBox').value += 'Original Transcription: \\n{original_transcription}\\n\\nEnglish Translation: \\n{english_translation}';
        document.getElementById('detectedLanguageBox').innerText = `{detected_language_name}`;
    """)
    # Generate and play audio for the transcription
    generate_and_play_audio(original_transcription)

  except Exception as e:
        print(f"Error in processing audio: {str(e)}")

# Function to save transcription text from the text box
def save_text(transcription_text):
    try:
        # Save the transcription text to a file (append if file exists)
        with open("transcription.txt", "a") as f:
            f.write(transcription_text + '\n')  # Add a new line after each save
        print("Text saved successfully.")
    except Exception as e:
        print(f"Error saving text: {str(e)}")

# Audio Evaluation

Speech to Text (STT) system evaluation (Run this code after transcription)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from difflib import SequenceMatcher

def calculate_metrics(reference_text, predicted_text):
    """
    Calculate evaluation metrics for a speech-to-text system.

    Args:
        reference_text (str): The ground truth text.
        predicted_text (str): The transcription output.

    Returns:
        dict: A dictionary containing accuracy, precision, recall, and F1 score.
    """
    # Tokenize the reference and predicted texts
    reference_tokens = reference_text.lower().split()
    predicted_tokens = predicted_text.lower().split()

    # Align tokens using SequenceMatcher to simulate true positives, false positives, and false negatives
    matcher = SequenceMatcher(None, reference_tokens, predicted_tokens)
    matches = matcher.get_matching_blocks()

    # Calculate true positives, false positives, and false negatives
    true_positives = sum(m.size for m in matches[:-1])  # Ignore the last match (dummy match)
    false_positives = len(predicted_tokens) - true_positives
    false_negatives = len(reference_tokens) - true_positives

    # Precision, recall, and F1 score calculations
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Accuracy: Proportion of correctly predicted words
    accuracy = true_positives / len(reference_tokens) if len(reference_tokens) > 0 else 0

    # Return the metrics as a dictionary
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }
    pass


In [ ]:
if __name__ == "__main__":
    try:
        # Step 1: Define the reference text (ground truth)
        reference_text = "I love programming and building AI systems."

        # Step 2: Read the transcription text from the file
        transcription_file_path = "/content/transcription_0.txt"
        with open(transcription_file_path, "r") as file:
            transcription = file.read().strip()  # Read and strip any extra whitespace

        # Step 3: Calculate metrics
        metrics = calculate_metrics(reference_text, transcription)

        # Step 4: Print the evaluation metrics
        print("Evaluation Metrics:")
        for metric, value in metrics.items():
            print(f"{metric.capitalize()}: {value:.2f}")

    except Exception as e:
        print(f"Failed to process transcription: {e}")

Text to Speech (TTS) Evaluation

In [ ]:
import whisper

# Function to calculate precision, recall, and F1 score
def calculate_token_metrics(reference_text, transcription):
    """
    Calculates precision, recall, and F1 score based on token-level comparison.

    Args:
        reference_text (str): The ground truth transcription.
        transcription (str): The generated transcription to compare with the ground truth.

    Returns:
        dict: Evaluation metrics (precision, recall, F1 score).
    """
    # Split the reference and transcription into tokens (words)
    reference_tokens = reference_text.split()
    transcription_tokens = transcription.split()

    # Calculate true positives (TP), false positives (FP), and false negatives (FN)
    true_positives = len(set(reference_tokens).intersection(transcription_tokens))
    false_positives = len(set(transcription_tokens) - set(reference_tokens))
    false_negatives = len(set(reference_tokens) - set(transcription_tokens))

    # Calculate precision, recall, and F1 score
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Accuracy: Proportion of correctly predicted words
    accuracy = true_positives / len(reference_tokens) if len(reference_tokens) > 0 else 0

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }

# Step 1: Define the reference text (ground truth)
reference_text = "I love programming and building AI systems."

# Load the Whisper model
model = whisper.load_model("base")

# Step 2: Transcribe the audio file
result = model.transcribe("/content/recording_1.wav")
transcription = result["text"]

# Save the transcription to a text file
with open("transcription_test.txt", "w") as text_file:
    text_file.write(transcription)

# Step 3: Read the content of the transcription file into the 'transcription' variable
with open("transcription_test.txt", 'r') as f:
    transcription = f.read()

# Step 4: Calculate metrics
metrics = calculate_token_metrics(reference_text, transcription)

# Step 5: Print the evaluation metrics
print("Evaluation Metrics:")
for metric, value in metrics.items():
    print(f"{metric.capitalize()}: {value:.2f}")


Latency calculation for STT

In [ ]:
import time
import whisper

def calculate_latency(audio_file_path):
    try:
        # Load the Whisper model (choose appropriate model size)
        model = whisper.load_model("medium")

        # Start timer before transcription
        start_time = time.time()

        # Perform transcription
        result = model.transcribe(audio_file_path)

        # Stop timer after transcription is completed
        end_time = time.time()

        # Calculate the latency (time taken for transcription)
        latency = end_time - start_time

        # Print the result (transcription and latency)
        print(f"Transcription: {result['text']}")
        print(f"Latency: {latency:.2f} seconds")

        return latency
    except Exception as e:
        print(f"Error in transcription: {e}")
        return None

# Example usage
audio_file_path = "/content/recording_0.wav"  # Replace with your audio file path
latency = calculate_latency(audio_file_path)
if latency:
    print(f"Transcription completed with latency of {latency:.2f} seconds.")
else:
    print("Failed to transcribe audio.")


Latency calculation for Google TTS

In [ ]:
import time
from gtts import gTTS
import os

def calculate_gtts_latency(text, lang="en"):
    try:
        # Start timer before text-to-speech conversion
        start_time = time.time()

        # Perform text-to-speech conversion using gTTS
        tts = gTTS(text=text, lang="ur")

        # Save the speech audio to a file
        audio_file_path = "output_audio.mp3"  # You can specify the file path here
        tts.save(audio_file_path)

        # Stop timer after the audio file is saved
        end_time = time.time()

        # Calculate the latency (time taken for text-to-speech conversion)
        latency = end_time - start_time

        # Print the result (audio file path and latency)
        print(f"Generated speech saved to: {audio_file_path}")
        print(f"Latency: {latency:.2f} seconds")

        return latency
    except Exception as e:
        print(f"Error in text-to-speech conversion: {e}")
        return None

# Function to read text from a file
def read_text_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            text = file.read()  # Read the entire file content
        return text
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

# Example usage
file_path = "/content/transcription_0.txt"  # Path to your text file
text = read_text_from_file(file_path)
print(text)
latency = calculate_gtts_latency(text)
if latency:
    print(f"Text-to-speech conversion completed with latency of {latency:.2f} seconds.")
else:
    print("Failed to convert text to speech.")


In [ ]:
'''from gtts import gTTS
from IPython.display import Audio

# Read the text from a file
with open('/content/transcription.txt', 'r') as file:
    text = file.read()

# Convert the text to speech
tts = gTTS(text, lang="ur")
tts.save('1.wav')

# Play the generated audio
sound_file = '1.wav'
Audio(sound_file, autoplay=True)'''

To see the supported languages by Whisper, use the following code.

In [ ]:
'''import whisper

# Get Whisper's supported languages
supported_languages = whisper.tokenizer.LANGUAGES
for code, name in supported_languages.items():
    print(f"{code}: {name}")
'''

# Display

In [ ]:
from google.colab import output
output.register_callback('save_and_process_image', save_and_process_image)
output.register_callback('train_model', train_model)
output.register_callback('pred', pred)
output.register_callback('notebook.save_audio', save_audio)
output.register_callback('notebook.save_text', save_text)

In [ ]:
# Display the HTML code in Google Colab
display(HTML(html_code))

In [ ]:
#try this is gtts isn't working
#!pip install gTTS sounddevice soundfile numpy

In [ ]:
#try installing this if facing issue with sound
#!nvidia-smi

A quick way to check the transcription via whisper api

In [ ]:
'''import whisper

model = whisper.load_model("turbo")
result = model.transcribe("/content/Audio/that-s-true-japanese.wav")
print(result["text"])'''

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


そうですね。
